In [1]:
import os
import cv2
import utils
import numpy as np
from numpy import genfromtxt
from keras.models import load_model
from keras.utils import CustomObjectScope
import tensorflow as tf
with CustomObjectScope({'tf': tf}):
  model = load_model('model/nn4.small2.v1.h5')

Using TensorFlow backend.
/Users/macworld/opt/anaconda3/envs/py2/lib/python2.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
weights = utils.weights
weight_dict = utils.load_weights()

In [3]:
for name in weights:
    model.get_layer(name).set_weights(weight_dict[name])

In [4]:
from __future__ import print_function
from email.mime.text import MIMEText
import base64
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def send_mail():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'creds.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    '''results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])'''
    return(service)



def create_message(sender, to, subject, message_text):
  """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text)
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  return {'raw': base64.urlsafe_b64encode(message.as_string())}

def send_message(service, user_id, message):
  """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print('Message Id: %s' % message['id'])
    return message
  except():
    print('An error occurred: %s' % error)

### Converting image to embedding

In [5]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

### function to recognise image

In [90]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        #print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.30:
        return str(name)
    else:
        return None

### Capturing image samples

In [7]:
def sampling():
    mid = raw_input("Enter the mailid")
    cam = cv2.VideoCapture(0)

    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    fn = 0
    count = 0
    while(True):
        ret, img = cam.read()
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
            count += 1
            # Save the captured image into the datasets folder
            cv2.imwrite("images/"+mid + ".jpg", img[y1:y2,x1:x2])
            cv2.imshow('image', img)
        k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 10: # Take 30 face sample and stop video
             break
    cam.release()
    cv2.destroyAllWindows()

## This check if the gien creds is the person or not

In [8]:
def mail_details():
    a = raw_input("Enter your mail id")
    b = raw_input("Enter the reciever mail id")
    c = raw_input("Enter the mail subject")
    d = raw_input("Enter the mail body")
    return([a,b,c,d])

In [43]:
def check_credentials(id1,id2,count,fn):
    if(id1 == id2):
        count += 1
        print("hello",id1)
    else:
        fn += 1
        print("You are not who you say you are")
    if(count >3):
        print("sending mail")
        service = send_mail()
        from_mail,to_mail,subject,content = mail_details()
        txt = create_message(from_mail,to_mail,subject,content)
        send_message(service,from_mail,txt)
        #txt = create_message('kajalsinha01@gmail.com','rohit123sinha456@gmail.com','Testing from anaconda','This workd')
        #send_message(service,'kajalsinha01@gmail.com',txt)
        return(10,fn)
    return(count,fn)

In [51]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings,givenid):
    fn = 0
    count = 0
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #while(count > 5):
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            #print(identity)
            #print("calling creds")
            count,fn = check_credentials(identity,givenid,count,fn)
            if(count == 10):
                print(fn)
                break
            '''
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)
        '''
        key = cv2.waitKey(100)
        if(count == 10):
            break
        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Lets Run the program

In [92]:
mailid = raw_input("Enter mail id")

Enter mail idpersonrohit02@gmail.com


In [93]:
%%time
try:
    input_embeddings = create_input_image_embeddings()
    recognize_faces_in_cam(input_embeddings,mailid)
except:
    pass

You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
You are not who you say you are
CPU times: user 20.3 s, sys: 1.6 s, total: 21.9 s
Wall time: 10.2 s


In [22]:
%%time
sampling()

Enter the mailidpersonrohit02@gmail.com
CPU times: user 10.7 s, sys: 2.13 s, total: 12.9 s
Wall time: 57 s


In [142]:
a = 0
def incre(a):
    return([12,23,34,45])

In [143]:
q,w,e,r = incre(a)

In [15]:
%%time
print("hello")

hello
CPU times: user 2.69 ms, sys: 4.74 ms, total: 7.43 ms
Wall time: 5.73 ms
